In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import os
import numpy as np
import pyspark.sql.functions as func

In [2]:
sc = SparkContext()
sqlcontext = SQLContext(sc)

In [3]:
accommodation_df = sqlcontext.read.option("header","true").csv("data/accommodation_code.csv")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46308)
Traceback (most recent call last):
  File "/home

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46308)
Traceback (most recent call last):
  File "/home/hadoop/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-55e208bfdfaa>", line 1, in <module>
    accommodation_df = sqlcontext.read.option("header","true").csv("data/accommodation_code.csv")
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/python/pyspark/sql/readwriter.py", line 410, in csv
    return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/home/hadoop/spark-2.2.2-bin-hadoop2.7/pytho

Py4JError: An error occurred while calling o22.csv

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.functions import isnan
from pyspark.sql.functions import min, max, col

In [ ]:

# Adress에서 정규식으로 동 뽑기
#t = accommodation_df.withColumn('Dong', regexp_extract('Address', '\\((.*?)\\)', 1))
t = accommodation_df.withColumn('Dong', regexp_extract('adm_code', '(^[0-9]{8})', 1))
t.show(10, truncate = False)

In [6]:
df = t.groupBy("Dong").agg(count('Dong'))

In [7]:
dong_df = sqlcontext.read.option("header","true").csv("data/dongCode.csv")

In [8]:
dong_df.show()

+--------+--------+---+----+-----------+
|통계청행정동코드|행자부행정동코드|시도명|시군구명|       행정동명|
+--------+--------+---+----+-----------+
| 1101053|11110530| 서울| 종로구|        사직동|
| 1101054|11110540| 서울| 종로구|        삼청동|
| 1101055|11110550| 서울| 종로구|        부암동|
| 1101056|11110560| 서울| 종로구|        평창동|
| 1101057|11110570| 서울| 종로구|        무악동|
| 1101058|11110580| 서울| 종로구|        교남동|
| 1101060|11110600| 서울| 종로구|        가회동|
| 1101061|11110615| 서울| 종로구|종로1.2.3.4가동|
| 1101063|11110630| 서울| 종로구|    종로5.6가동|
| 1101064|11110640| 서울| 종로구|        이화동|
| 1101067|11110670| 서울| 종로구|       창신1동|
| 1101068|11110680| 서울| 종로구|       창신2동|
| 1101069|11110690| 서울| 종로구|       창신3동|
| 1101070|11110700| 서울| 종로구|       숭인1동|
| 1101071|11110710| 서울| 종로구|       숭인2동|
| 1101072|11110515| 서울| 종로구|      청운효자동|
| 1101073|11110650| 서울| 종로구|        혜화동|
| 1102052|11140520| 서울|  중구|        소공동|
| 1102054|11140540| 서울|  중구|        회현동|
| 1102055|11140550| 서울|  중구|         명동|
+--------+--------+---+----+-----------+
only showing top

In [8]:
joined = df.join(dong_df, df.Dong == dong_df.행자부행정동코드, how='right')

In [10]:
joined.select(["Dong", '행자부행정동코드']).show()

+--------+--------+
|    Dong|행자부행정동코드|
+--------+--------+
|11110530|11110530|
|    null|11110540|
|11110550|11110550|
|11110560|11110560|
|    null|11110570|
|11110580|11110580|
|    null|11110600|
|11110615|11110615|
|11110630|11110630|
|    null|11110640|
|11110670|11110670|
|11110680|11110680|
|    null|11110690|
|11110700|11110700|
|11110710|11110710|
|    null|11110515|
|11110650|11110650|
|11140520|11140520|
|11140540|11140540|
|11140550|11140550|
+--------+--------+
only showing top 20 rows



In [11]:
total = joined.count()
null_count = joined.filter((joined["행자부행정동코드"] == "") |\
                           joined["행자부행정동코드"].isNull() |\
                           isnan(joined["행자부행정동코드"])).count()
print(total,null_count)
joined.filter((joined["행자부행정동코드"] == "") |\
              joined["행자부행정동코드"].isNull() |\
              isnan(joined["행자부행정동코드"]))\
              .select(["Dong", '행자부행정동코드'])\
              .fillna(0, subset=['count(Dong)'])\
              .show()

424 0
+----+--------+
|Dong|행자부행정동코드|
+----+--------+
+----+--------+



In [9]:
joined.fillna(0, subset=['count(Dong)']).show()
result = joined.fillna(0, subset=['count(Dong)'])
print(joined.count())

+--------+-----------+--------+--------+---+----+-----------+
|    Dong|count(Dong)|통계청행정동코드|행자부행정동코드|시도명|시군구명|       행정동명|
+--------+-----------+--------+--------+---+----+-----------+
|11110530|          7| 1101053|11110530| 서울| 종로구|        사직동|
|    null|          0| 1101054|11110540| 서울| 종로구|        삼청동|
|11110550|          2| 1101055|11110550| 서울| 종로구|        부암동|
|11110560|          2| 1101056|11110560| 서울| 종로구|        평창동|
|    null|          0| 1101057|11110570| 서울| 종로구|        무악동|
|11110580|          1| 1101058|11110580| 서울| 종로구|        교남동|
|    null|          0| 1101060|11110600| 서울| 종로구|        가회동|
|11110615|        105| 1101061|11110615| 서울| 종로구|종로1.2.3.4가동|
|11110630|         28| 1101063|11110630| 서울| 종로구|    종로5.6가동|
|    null|          0| 1101064|11110640| 서울| 종로구|        이화동|
|11110670|         31| 1101067|11110670| 서울| 종로구|       창신1동|
|11110680|          4| 1101068|11110680| 서울| 종로구|       창신2동|
|    null|          0| 1101069|11110690| 서울| 종로구|       창신3동|
|1111070

In [11]:
_min, _max = result.select(min("count(Dong)"), max("count(Dong)")).first()
print(_min, _max)
save_df = result.withColumn("scale", (col("count(Dong)") - _min) / (_max - _min))

0 105


In [14]:
_df = save_df.sort("행자부행정동코드").select(["행자부행정동코드", "scale", "count(Dong)"])
_df.show()
_df.write.csv("0611_t3")

+--------+--------------------+-----------+
|행자부행정동코드|               scale|count(Dong)|
+--------+--------------------+-----------+
|11110515|                 0.0|          0|
|11110530| 0.06666666666666667|          7|
|11110540|                 0.0|          0|
|11110550| 0.01904761904761905|          2|
|11110560| 0.01904761904761905|          2|
|11110570|                 0.0|          0|
|11110580|0.009523809523809525|          1|
|11110600|                 0.0|          0|
|11110615|                 1.0|        105|
|11110630| 0.26666666666666666|         28|
|11110640|                 0.0|          0|
|11110650| 0.06666666666666667|          7|
|11110670| 0.29523809523809524|         31|
|11110680|  0.0380952380952381|          4|
|11110690|                 0.0|          0|
|11110700| 0.02857142857142857|          3|
|11110710|  0.3047619047619048|         32|
|11140520| 0.22857142857142856|         24|
|11140540|                 0.4|         42|
|11140550|  0.6190476190476191| 

In [15]:
sc.textFile("hdfs://master:9000/user/hadoop/0611_t3/part*").coalesce(1).saveAsTextFile("hdfs://master:9000/user/hadoop/0611_t3_")